In [22]:
import pandas as pd
import numpy as np
import processing.b_feats.util
import processing.processing_utils
import importlib
from constants import LEVELS, START, MAX_DELAY
from rlenv.time_triggers import *
from compress_pickle import dump, load
import sys
import os
os.chdir('A:/ebay/')
sys.path.append('repo/processing/b_feats/test/')
import meta_time_feats_test  
importlib.reload(meta_time_feats_test)
importlib.reload(processing.b_feats.time_funcs)
importlib.reload(processing.b_feats.util)
setup_complex_lstg = meta_time_feats_test.setup_complex_lstg
delay_appendix_1 =  meta_time_feats_test.delay_appendix_1
delay_appendix_2 = meta_time_feats_test.delay_appendix_2
add_event = meta_time_feats_test.add_event
NEW_LSTG = 'new_lstg'
ACCEPTANCE = 'accept'
EXPIRE_LSTG = 'expire_lstg'
CENSORED = 'censored'

In [ ]:
events = setup_complex_lstg(None, meta=1, leaf=1)

events.index = events.index.droplevel(['cndtn'])
feats = processing.b_feats.util.get_cat_feats(events, levels=['meta'], feat_ind=4)

In [3]:
events = setup_complex_lstg(None, meta=1, leaf=1)

# add a group where there are 2 lstgs and no threads
offer = {
    'lstg': 23,
    'clock': 300
}
events = add_event(events, offer, trigger_type=NEW_LSTG, meta=1, leaf=2)
offer['clock'] = 325
events = add_event(events, offer, trigger_type=EXPIRE_LSTG, meta=1, leaf=2)

offer = {
    'lstg': 24,
    'clock': 350
}
events = add_event(events, offer, trigger_type=NEW_LSTG, meta=1, leaf=2)
offer['clock'] = 355
events = add_event(events, offer, trigger_type=EXPIRE_LSTG, meta=1, leaf=2)

# add a group where there are 2 lstgs, several threads, and 1 bin
offer = {
    'lstg': 25,
    'clock': 360
}
events = add_event(events, offer, trigger_type=NEW_LSTG, meta=1, leaf=3)
offer['clock'] = 365
offer['byr'] = True
offer['price'] = 35
offer['thread'] = 1
events = add_event(events, offer, trigger_type=OFFER, meta=1, leaf=3)

offer['price'] = 45
offer['clock'] = 370
offer['thread'] = 2
events = add_event(events, offer, trigger_type=OFFER, meta=1, leaf=3)

offer['price'] = 55
offer['clock'] = 375
offer['thread'] = 3
events = add_event(events, offer, trigger_type=OFFER, meta=1, leaf=3)

offer['price'] = 60
offer['clock'] = 380
offer['thread'] = 4
events = add_event(events, offer, trigger_type=OFFER, meta=1, leaf=3)

offer['clock'] = 385
offer['lstg'] = 26
events = add_event(events, offer, trigger_type=NEW_LSTG, meta=1, leaf=3)

offer['thread'] = 1
offer['price'] = 50
offer['clock'] = 387
offer['byr'] = True
events = add_event(events, offer, trigger_type=OFFER, meta=1, leaf=3)

offer['thread'] = 2
offer['price'] = 100
offer['clock'] = 389
events = add_event(events, offer, trigger_type=ACCEPTANCE,  meta=1, leaf=3)
offer['thread'] = 1
offer['byr'] = False
events = add_event(events, offer, trigger_type=CENSORED,  meta=1, leaf=3)

offer['clock'] = 400
offer['byr'] = False
offer['lstg'] = 25
offer['price'] = 60
offer['thread'] = 4
events = add_event(events, offer, trigger_type=ACCEPTANCE,  meta=1, leaf=3)
offer['thread'] = 3
events = add_event(events, offer, trigger_type=CENSORED,  meta=1, leaf=3)
offer['thread'] = 2
events = add_event(events, offer, trigger_type=CENSORED,  meta=1, leaf=3)
offer['thread'] = 1
events = add_event(events, offer, trigger_type=CENSORED,  meta=1, leaf=3)

events.byr = events.byr.astype(bool)
events.reject = events.reject.astype(bool)
events.accept = events.accept.astype(bool)
events.censored = events.censored.astype(bool)
events.flag = events.flag.astype(bool)
events.price = events.price.astype(np.int64)
events.clock = events.clock.astype(np.int64)

events.index = events.index.droplevel(['cndtn'])
feats = processing.b_feats.util.get_cat_feats(events, levels=['meta'], feat_ind=4)

C:\Users\navba\Anaconda3\envs\ebay\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [4]:
feats

,meta_slr_delay_25,meta_slr_delay_50,meta_slr_delay_75,meta_slr_delay_100,meta_byr_delay_25,meta_byr_delay_50,meta_byr_delay_75,meta_byr_delay_100,meta_slr_expire,meta_byr_expire
lstg,,,,,,,,,,
1,0.000058,0.000116,0.000116,0.000174,0.000004,0.000004,0.000006,0.000017,0.0,0.0
2,0.000058,0.000058,0.000116,0.000174,0.000004,0.000008,0.000017,0.000025,0.0,0.0
3,0.000058,0.000087,0.000116,0.000174,0.000004,0.000004,0.000008,0.000025,0.0,0.0
4,0.000058,0.000116,0.000116,0.000174,0.000004,0.000006,0.000012,0.000025,0.0,0.0
5,0.000058,0.000087,0.000116,0.000174,0.000004,0.000006,0.000012,0.000025,0.0,0.0
6,0.000058,0.000116,0.000116,0.000174,0.000004,0.000004,0.000008,0.000025,0.0,0.0
7,0.000058,0.000116,0.000116,0.000174,0.000004,0.000006,0.000012,0.000025,0.0,0.0
8,0.000058,0.000087,0.000116,0.000174,0.000004,0.000006,0.000012,0.000025,0.0,0.0
9,0.000058,0.000116,0.000116,0.000174,0.000004,0.000004,0.000012,0.000025,0.0,0.0


In [ ]:
.000012 * MAX_DELAY['byr']

In [ ]:
test = np.array([curr / MAX_DELAY['byr'] for curr in [5, 15, 50]])

In [ ]:
np.nanquantile(test, q=.25, interpolation='lower')

In [ ]:
events = setup_complex_lstg(None, meta=1, leaf=1)

# add a group where there are 2 lstgs and no threads
offer = {
    'lstg': 23,
    'clock': 300
}
events = add_event(events, offer, trigger_type=NEW_LSTG, meta=3, leaf=1)
offer['clock'] = 325
events = add_event(events, offer, trigger_type=EXPIRE_LSTG, meta=3, leaf=1)

offer = {
    'lstg': 24,
    'clock': 350
}
events = add_event(events, offer, trigger_type=NEW_LSTG, meta=3, leaf=1)
offer['clock'] = 355
events = add_event(events, offer, trigger_type=EXPIRE_LSTG, meta=3, leaf=1)

# add a group where there are 2 lstgs, several threads, and 1 bin
offer = {
    'lstg': 25,
    'clock': 360
}
events = add_event(events, offer, trigger_type=NEW_LSTG, meta=4, leaf=1)
offer['clock'] = 365
offer['byr'] = True
offer['price'] = 35
offer['thread'] = 1
events = add_event(events, offer, trigger_type=OFFER, meta=4, leaf=1)

offer['price'] = 45
offer['clock'] = 370
offer['thread'] = 2
events = add_event(events, offer, trigger_type=OFFER, meta=4, leaf=1)

offer['price'] = 55
offer['clock'] = 375
offer['thread'] = 3
events = add_event(events, offer, trigger_type=OFFER, meta=4, leaf=1)

offer['price'] = 60
offer['clock'] = 380
offer['thread'] = 4
events = add_event(events, offer, trigger_type=OFFER, meta=4, leaf=1)

offer['clock'] = 385
offer['lstg'] = 26
events = add_event(events, offer, trigger_type=NEW_LSTG, meta=4, leaf=1)

offer['thread'] = 1
offer['price'] = 50
offer['clock'] = 387
offer['byr'] = True
events = add_event(events, offer, trigger_type=OFFER, meta=4, leaf=1)

offer['thread'] = 2
offer['price'] = 100
offer['clock'] = 389
events = add_event(events, offer, trigger_type=ACCEPTANCE, meta=4, leaf=1)
offer['thread'] = 1
offer['byr'] = False
events = add_event(events, offer, trigger_type=CENSORED, meta=4, leaf=1)

offer['clock'] = 400
offer['byr'] = False
offer['lstg'] = 25
offer['price'] = 60
offer['thread'] = 4
events = add_event(events, offer, trigger_type=ACCEPTANCE, meta=4, leaf=1)
offer['thread'] = 3
events = add_event(events, offer, trigger_type=CENSORED, meta=4, leaf=1)
offer['thread'] = 2
events = add_event(events, offer, trigger_type=CENSORED, meta=4, leaf=1)
offer['thread'] = 1
events = add_event(events, offer, trigger_type=CENSORED, meta=4, leaf=1)
events.index = events.index.droplevel(['cndtn', 'leaf'])
events.byr = events.byr.astype(bool)
events.reject = events.reject.astype(bool)
events.accept = events.accept.astype(bool)
events.censored = events.censored.astype(bool)
events.flag = events.flag.astype(bool)
events.price = events.price.astype(np.int64)
events.clock = events.clock.astype(np.int64)

In [ ]:
clock_df = events.clock.unstack()
delay = pd.DataFrame(index=clock_df.index)
delay[0] = 0
for i in range(1, 8):
    if i in clock_df.columns:
        delay[i] = clock_df[i] - clock_df[i-1]
        if i in [2, 4, 6]:
            max_delay = MAX_DELAY['slr']
            censored = delay[i] > MAX_DELAY['slr']
        elif i != 7:
            censored = delay[i] > MAX_DELAY['byr']
            max_delay = MAX_DELAY['byr']
        else:
            censored = delay.index
            max_delay = 1
        delay.loc[censored, i] = np.NaN
        delay[i] = delay[i] / max_delay
events['delay'] = delay.rename_axis('index', axis=1).stack()

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(events['delay'])

In [ ]:
feats = processing.b_feats.util.get_cat_feats(events, levels=['meta'], feat_ind=3)
feats.isna().any().any()

In [ ]:
feats

In [ ]:
actual

In [ ]:
total_lstgs_index = pd.MultiIndex.from_tuples([(1, 2), (1, 1), (2, 3), (2, 1)], names=['meta','leaf'])
total_lstgs = pd.DataFrame(index=total_lstgs_index, data={'tot': [4, 3, 1, 9]})
lstg_index = pd.MultiIndex.from_tuples([(1, 2, 1), (1, 2, 2), (1, 2, 3), (1, 1, 6), (1, 1, 7), (1, 1, 8), (1, 1, 9), (2, 3, 1), (2, 1, 3), (2, 1, 2)], names=['meta', 'leaf', 'lstg'])
lstg_index

In [ ]:
total_lstgs.reindex(lstg_index)

In [ ]:
total_lstgs_index = pd.Index([1, 2])
total_lstgs = pd.DataFrame(index=total_lstgs_index, data={'tot': [4, 3]})
lstg_index = pd.MultiIndex.from_tuples([(1, 1), (1, 2), (1, 3), (2, 3), (2, 2)], names=['meta', 'lstg'])
lstg_index

In [ ]:
total_lstgs.reindex(lstg_index, level='meta')

In [ ]:
t = pd.Index([1, 2, 3, 4])
p = pd.Index([2, 1, 3, 4])

In [ ]:
t.equals(p)

In [ ]:
t = pd.DataFrame(data={'clock': [4, 9, 11, 1, 2, 15, 15], 'censored': [2, 2, 2, 2, 2, 2, 1]})

In [ ]:
t.sort_values(['clock', 'censored'])

In [5]:
path = 'A:/ebay/data/chunks/m1.gz'
d = load(path)
L, T, O = [d[k] for k in ['listings', 'threads', 'offers']]

In [10]:
def check_bounds(L, col):
    print('max: {}'.format(L[col].max()))
    print('min: {}'.format(L[col].min()))

In [11]:
L.columns

Index(['meta', 'leaf', 'cndtn', 'start_date', 'end_time', 'start_price',
       'arrival_rate', 'flag'],
      dtype='object')

In [12]:
check_bounds(L, 'arrival_rate')

max: 0.9999998982781346
min: 1.0172186534179498e-08


In [13]:
check_bounds(L, 'start_price')

max: 21000000.0
min: 0.01


In [14]:
O.columns

Index(['clock', 'price', 'accept', 'reject', 'censored'], dtype='object')

In [15]:
T.columns

Index(['start_time', 'byr_hist', 'bin'], dtype='object')

In [17]:
check_bounds(T, 'byr_hist')

max: 0.9998214528123667
min: 2.8399425422824846e-08


In [24]:
levels = ['meta', 'leaf', 'cndtn']
events = processing.b_feats.util.create_events(L, T, O, levels)

In [19]:
events.columns

Index(['accept', 'censored', 'clock', 'price', 'reject', 'byr', 'flag',
       'start_price'],
      dtype='object')

In [23]:
events = setup_complex_lstg(None, meta=1, leaf=1)
events = setup_complex_lstg(events, meta=2, leaf=1, starter=11)
events.index = events.index.droplevel(['cndtn', 'leaf'])
feats = processing.b_feats.util.get_cat_feats(events, levels=['meta'], feat_ind=4)

In [26]:
events.isna().any()

accept          False
byr_hist         True
censored        False
clock           False
price            True
reject          False
byr             False
flag            False
start_price     False
arrival_rate    False
dtype: bool

In [28]:
print((events.loc[events.byr_hist.isna()].index.get_level_values('thread') != 0).any())

False


In [29]:
print((events.loc[events.price.isna()].index.get_level_values('thread') != 0).any())

False


In [30]:
np.nanquantile(np.array([.4, .3, .6, .65, .7, .45, .35, .55, .8, .9]), q=.25, interpolation='lower')

0.4

In [ ]:
[0.35 0.4  0.4  0.35 0.35 0.35 0.35 0.4  0.35 0.35 0.35]